In [8]:
import glob
import itertools as it
import pandas as pd
#import mako

def get_secs(timestring):
    # convert h:mm:ss or m:ss to seconds
    parts = map(float,timestring.split(":")[::-1])
    products = [60 ** i for i in xrange(len(parts))]
    total = sum(part * product for part,product in it.izip(parts, products))
    return total

def parse_filename(filename, d):
    filename_fields = filename.split("_")
    d["dataset"] = filename_fields[0]
    d["k"] = int(filename_fields[1][1:])
    d["frequency"] = int(filename_fields[2][1:])
    return d

def parse_info(filename, d):
    with open(filename) as f:
        d["dsk size"] = float(f.readline().split()[4][:-1])
    return d

def parse_bench(filename, d):
    order = "variable" if "varord" in filename else "fixed"
    d[order] = d[order] if order in d else {}
    with open(filename) as f:
        lines = f.readlines()
        size_fields = ["DBG size", "LCS size"]
        op_fields = ["forward", "backward", "lastchar", "shorter_1", "shorter_2", "shorter_4", "shorter_8",
                       "longer_1", "longer_2", "longer_4", "longer_8", "maxlen ", "maxlen*"]
        for i,line in enumerate(lines):
            for field in size_fields + op_fields:
                if field in line: 
                    factor = 1
                    if field in op_fields:
                        factor = 0.001 # convert to microsec if time measurement
                    d[order][field.rstrip()] = float(line.split()[-2]) * factor
                    if "longer" in field:
                        d[order][field + " per node"] = float(lines[i+1].split()[3]) * factor
                        d[order][field + " nodes"] = int(lines[i+1].split()[-1][1:-1]) * factor
        d[order]["Total size"] = d[order]["DBG size"] + (d[order]["LCS size"] if order == "variable" else 0)
    return d

def parse_build(filename, d):
    order = "variable" if "varord" in filename else "fixed"
    d[order] = d[order] if order in d else {}
    
    if "peak disk" not in d[order]: d[order]["peak disk"] = []
    if "peak mem" not in d[order]: d[order]["peak mem"] = []
    if "wall time" not in d[order]: d[order]["wall time"] = []
        
    fields = ["wall clock", "Peak disk allocs"]
    with open(filename) as f:
        for line in f:
            if "Building from" in line: d["# kmers"] = int(line.split()[6])/2
            elif "Peak disk allocs" in line: d[order]["peak disk"].append(float(line.split()[-2])/1024) # MB -> GB
            elif "Maximum resident" in line: d[order]["peak mem"].append(float(line.split()[-1])/1024**2) # KB -> GB
            elif "wall clock" in line: d[order]["wall time"].append(get_secs(line.split()[-1])/60.0)
    return d

def parse(filename, d=None):
    d = d or dict()
    parse_filename(filename, d)
    if "info" in filename: parse_info(filename, d)
    elif "bench" in filename: parse_bench(filename, d)
    elif "build" in filename: parse_build(filename, d)
    return d

def parse_group(files, d=None):
    for f in files:
        d = parse(f, d)
    return d

def generate_table_rows(d):
    # output two runs of experiments, fixed and variable, with averages of peak disk, peak mem, wall time
    for order in ["fixed", "variable"]:
        for i in xrange(len(d[order]["peak disk"])):
            new_d = {"order": order}
            for key, data in d.items():
                if key not in ["peak disk", "peak mem", "wall time", "fixed", "variable"]: new_d[key] = data
            for key, data in d[order].items():
                if key not in ["peak disk", "peak mem", "wall time", "fixed", "variable"]: new_d[key] = data
            for x in ["peak disk", "peak mem", "wall time"]:
                new_d[x] = d[order][x][i]
            yield new_d

def get_dataframe(groups):
    return pd.DataFrame(list(it.chain.from_iterable(generate_table_rows(parse_group(group)) for group in groups)))
            
def get_files():
    log_files = [x for x in glob.glob("*.log") if "dsk" not in x] #ignore DSK timings for now
    return [list(g) for _,g in it.groupby(log_files, key=lambda x: "".join(x.split("_")[:3]))]

def get_data():
    groups = get_files()
    df = get_dataframe(groups)
    return df

if __name__ == "__main__":
    get_data()

In [22]:
df = get_data()

datasets = ["ecoli", "chrom14", "human", "parrot"]

pd.options.display.float_format = '{:.2f}'.format

# describe dsk data
# min should work for these fixed values
df.groupby(["dataset", "k"])[["dsk size", "k", "# kmers", "frequency"]].min().transpose()[datasets]

#df

dataset          ecoli      chrom14                                         \
k                   28           16           32           48           56   
dsk size          1.60         4.20         7.10         7.20         6.90   
k                28.00        16.00        32.00        48.00        56.00   
# kmers   204098902.00 561434000.00 951050150.00 482664928.00 461445333.00   
frequency         1.00         1.00         1.00         1.00         1.00   

dataset                        human        parrot  
k                   64            56            56  
dsk size          6.40         27.00         71.00  
k                64.00         56.00         56.00  
# kmers   427654657.00 1794522954.00 4716731435.00  
frequency         1.00          2.00          1.00

In [26]:
grouped = df.groupby(["dataset", "k", "order"])
fields = ["Total size", "wall time", "peak mem", "peak disk", "forward", "backward", "lastchar"] \
    + [col for col in list(df) if col.startswith("maxlen")] \
    + [col for col in list(df) if col.startswith("shorter")] \
    + sorted([col for col in list(df) if col.startswith("longer")],key=lambda x:x[::-1])
grouped.min()[fields].transpose()[datasets]

dataset            ecoli          chrom14                                  \
k                     28               16              32              48   
order              fixed variable   fixed variable  fixed variable  fixed   
Total size        162.65   420.14  584.59   778.11 885.68  1541.30 391.58   
wall time           3.93     5.09    6.85     8.79  13.42    18.36  13.53   
peak mem            3.16     3.16    2.78     2.78   3.21     3.21   3.23   
peak disk          12.17    12.17   33.47    33.47  56.69    56.69  57.54   
forward             6.00    17.03    6.68    25.72   5.66    18.49   6.29   
backward            8.23    59.77   10.48    90.15   8.57    65.27   8.50   
lastchar            0.01     0.01    0.01     0.01   0.01     0.01   0.01   
maxlen               nan     5.41     nan     6.49    nan     5.93    nan   
maxlen*              nan     1.43     nan     2.26    nan     2.16    nan   
shorter_1            nan    14.65     nan    12.49    nan    15.75    nan   
shorter_2            nan    14.83     nan    12.19    nan    15.88    nan   
shorter_4            nan    15.11     nan    11.76    nan    16.12    nan   
shorter_8            nan    15.73     nan    12.23    nan    16.53    nan   
longer_1             nan    21.53     nan    32.99    nan    22.27    nan   
longer_2             nan    56.96     nan   110.87    nan    58.73    nan   
longer_4             nan   503.60     nan  1465.82    nan   529.27    nan   
longer_8             nan  6441.33     nan     9.66    nan 10905.20    nan   
longer_1 per node    nan    10.82     nan     9.97    nan    11.74    nan   
longer_2 per node    nan     8.74     nan     7.05    nan     9.66    nan   
longer_4 per node    nan     7.98     nan     6.40    nan     8.70    nan   
longer_8 per node    nan     6.44     nan     8.55    nan     7.32    nan   
longer_1 nodes       nan    39.80     nan    66.18    nan    37.93    nan   
longer_2 nodes       nan   130.32     nan   314.36    nan   121.64    nan   
longer_4 nodes       nan  1262.84     nan  4577.98    nan  1216.52    nan   
longer_8 nodes       nan 20010.15     nan    22.59    nan 29807.51    nan   

dataset                                                      human           \
k                              56              64               56            
order             variable  fixed variable  fixed variable   fixed variable   
Total size         1271.49 413.94  1415.54 436.97  1562.59 1706.21  5547.89   
wall time            17.21  14.37    18.72  15.26    19.92   64.45    83.85   
peak mem              3.22   3.22     3.22   3.22     3.22    7.65     9.31   
peak disk            57.54  56.68    56.68  67.35    67.35  248.37   248.37   
forward              16.50   6.24    16.17   6.01    15.39    7.07    18.31   
backward             58.65   8.47    55.63   8.18    53.43    9.27    62.85   
lastchar              0.01   0.01     0.01   0.01     0.02    0.01     0.01   
maxlen                5.91    nan     5.98    nan     5.75     nan     6.71   
maxlen*               1.55    nan     1.56    nan     1.55     nan     2.02   
shorter_1            18.00    nan    17.72    nan    17.84     nan    19.54   
shorter_2            18.11    nan    17.79    nan    17.89     nan    19.68   
shorter_4            18.39    nan    18.02    nan    18.05     nan    19.90   
shorter_8            18.85    nan    18.39    nan    18.37     nan    20.29   
longer_1             20.48    nan    18.61    nan    17.84     nan    21.06   
longer_2             47.14    nan    41.08    nan    37.11     nan    49.01   
longer_4            397.52    nan   323.50    nan   263.10     nan   446.51   
longer_8           7007.46    nan  5338.38    nan  4844.09     nan 18349.80   
longer_1 per node    12.95    nan    12.38    nan    12.24     nan    13.25   
longer_2 per node    11.40    nan    11.24    nan    11.13     nan    11.74   
longer_4 per node    10.71    nan    10.59    nan    10.36     nan    11.04   
longer_8 per node     9.38

In [27]:
grouped.min()[fields].transpose()[datasets]["chrom14"]

k                     16              32              48              56  \
order              fixed variable  fixed variable  fixed variable  fixed   
Total size        584.59   778.11 885.68  1541.30 391.58  1271.49 413.94   
wall time           6.85     8.79  13.42    18.36  13.53    17.21  14.37   
peak mem            2.78     2.78   3.21     3.21   3.23     3.22   3.22   
peak disk          33.47    33.47  56.69    56.69  57.54    57.54  56.68   
forward             6.68    25.72   5.66    18.49   6.29    16.50   6.24   
backward           10.48    90.15   8.57    65.27   8.50    58.65   8.47   
lastchar            0.01     0.01   0.01     0.01   0.01     0.01   0.01   
maxlen               nan     6.49    nan     5.93    nan     5.91    nan   
maxlen*              nan     2.26    nan     2.16    nan     1.55    nan   
shorter_1            nan    12.49    nan    15.75    nan    18.00    nan   
shorter_2            nan    12.19    nan    15.88    nan    18.11    nan   
shorter_4            nan    11.76    nan    16.12    nan    18.39    nan   
shorter_8            nan    12.23    nan    16.53    nan    18.85    nan   
longer_1             nan    32.99    nan    22.27    nan    20.48    nan   
longer_2             nan   110.87    nan    58.73    nan    47.14    nan   
longer_4             nan  1465.82    nan   529.27    nan   397.52    nan   
longer_8             nan     9.66    nan 10905.20    nan  7007.46    nan   
longer_1 per node    nan     9.97    nan    11.74    nan    12.95    nan   
longer_2 per node    nan     7.05    nan     9.66    nan    11.40    nan   
longer_4 per node    nan     6.40    nan     8.70    nan    10.71    nan   
longer_8 per node    nan     8.55    nan     7.32    nan     9.38    nan   
longer_1 nodes       nan    66.18    nan    37.93    nan    31.64    nan   
longer_2 nodes       nan   314.36    nan   121.64    nan    82.72    nan   
longer_4 nodes       nan  4577.98    nan  1216.52    nan   742.10    nan   
longer_8 nodes       nan    22.59    nan 29807.51    nan 14945.17    nan   

k                              64           
order             variable  fixed variable  
Total size         1415.54 436.97  1562.59  
wall time            18.72  15.26    19.92  
peak mem              3.22   3.22     3.22  
peak disk            56.68  67.35    67.35  
forward              16.17   6.01    15.39  
backward             55.63   8.18    53.43  
lastchar              0.01   0.01     0.02  
maxlen                5.98    nan     5.75  
maxlen*               1.56    nan     1.55  
shorter_1            17.72    nan    17.84  
shorter_2            17.79    nan    17.89  
shorter_4            18.02    nan    18.05  
shorter_8            18.39    nan    18.37  
longer_1             18.61    nan    17.84  
longer_2             41.08    nan    37.11  
longer_4            323.50    nan   263.10  
longer_8           5338.38    nan  4844.09  
longer_1 per node    12.38    nan    12.24  
longer_2 per node    11.24    nan    11.13  
longer_4 per node    10.59    nan    10.36  
longer_8 per node     9.03    nan     9.16  
longer_1 nodes       30.06    nan    29.16  
longer_2 nodes       73.11    nan    66.70  
longer_4 nodes      610.77    nan   507.78  
longer_8 nodes    11824.04    nan 10571.74